In [1]:
from tweepy.streaming import StreamListener
from tweepy.auth import OAuthHandler
from tweepy import Stream

from datetime import datetime
import json

import pandas as pd
from pymongo import MongoClient

In [ ]:
CONSUMER_KEY= "${}"
CONSUMER_SECRET= "${}"
ACCESS_TOKEN= "${}"
ACCESS_TOKEN_SECRET= "${}"

In [9]:
consumer_key = "xxxxxx"
consumer_secret = "xxxxxx"
access_token = "xxxxxx"
access_token_secret = "xxxxxx"

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
#api = tweepy.API(auth)
#api.update_status('tweepy + oauth!')

In [4]:
class MyListener(StreamListener):
    def on_data(self,data):
        print("ayn")
        print(data)
        tweet = json.loads(data)
        created_at = tweet["created_at"]
        id_str = tweet[ "id_str"]
        text = tweet[ "text"]
        obj = {"created_at":created_at, "id_str":id_str, "text":text}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True
    
    def on_error(self, status):
        print(status)
        return False

In [5]:
mylistener = MyListener()
mylistener

In [6]:
mystream = Stream(auth, listener = mylistener)
mystream

In [7]:
client = MongoClient('localhost', 27017)
db = client.twitterdb
col = db.tweets
keywords = ['Hello']

In [8]:
print('inicia stream')
#mystream.filter(track=keywords)
mystream.filter(track=['bieber'])
print('fecha stream')

inicia stream
401
fecha stream


In [ ]:
class LimitedListener(StreamListener):

    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 10

    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)

In [ ]:
limited_listener = LimitedListener()
limited_stream = Stream(auth, listener = limited_listener)
print('inicia limited stream')
limited_stream.filter(track=keywords)
print('fecha limited stream')

In [ ]:
"""
client = MongoClient('localhost', 27017)
db_test = client.testdb
collection_test = db_test.col_test
obj_test = {"putetinho": "ayn ayn"}
collection_test.insert_one(obj_test)
"""

In [ ]:
col.find_one()

In [ ]:
dataset = [{"created_at":item["created_at"],
           "text":item["text"],
           } for item in col.find()]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]